In [7]:
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix # Importamos las herramientas de evaluación

# 1. Mostrar cómo se ve una fila ANTES de la transformación
print("--- ANTES DE LA TRANSFORMACIÓN ---")
print(df[['InternetService', 'Contract', 'PaymentMethod']].head(3))


# 2. Aplicar One-Hot Encoding
#    'drop_first=True' es una buena práctica para evitar redundancia en los datos
df_transformado = pd.get_dummies(df, drop_first=True)


# 3. Mostrar cómo se ven las mismas columnas DESPUÉS de la transformación
print("\n\n--- DESPUÉS DE LA TRANSFORMACIÓN ---")
# Seleccionamos las nuevas columnas que se crearon para ver el cambio
print(df_transformado[['InternetService_Fiber optic', 'InternetService_No', 'Contract_One year', 'Contract_Two year']].head(3))

# 4. Separar las características (X) de la variable objetivo (y)
#    Nuestra columna objetivo 'Churn_Yes' no es una característica, es lo que queremos predecir.
X = df_transformado.drop('Churn_Yes', axis=1) # Todas las columnas MENOS la de Churn
y = df_transformado['Churn_Yes'] # Solo la columna de Churn

# 2. Dividir los datos: 80% para entrenamiento, 20% para prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Imprimir el tamaño de cada conjunto para verificar
print("--- TAMAÑO DE LOS CONJUNTOS ---")
print(f"Características de entrenamiento (X_train): {X_train.shape}")
print(f"Características de prueba (X_test):          {X_test.shape}")
print(f"Objetivo de entrenamiento (y_train):    {y_train.shape}")
print(f"Objetivo de prueba (y_test):         {y_test.shape}")

# 1. Crear una instancia del modelo Random Forest
#    'n_estimators=100' significa que nuestro bosque tendrá 100 árboles.
#    'random_state=42' es para que nuestros resultados sean reproducibles.
modelo = RandomForestClassifier(n_estimators=100, random_state=42)

# 2. Entrenar el modelo con los datos de entrenamiento
print("Entrenando el modelo Random Forest... ¡Esto puede tardar unos segundos!")
modelo.fit(X_train, y_train)

print("\n¡Modelo entrenado exitosamente!")

# 1. Obtener las probabilidades de predicción en lugar de la clasificación final
#    predict_proba nos da dos columnas: [prob_de_No, prob_de_Yes]
probabilidades = modelo.predict_proba(X_test)

# 2. Nos quedamos solo con la probabilidad de que el cliente SÍ se vaya (la segunda columna)
prob_churn_yes = probabilidades[:, 1]

# 3. Definimos nuestro nuevo umbral. ¡Vamos a ser más sensibles!
nuevo_umbral = 0.20

# 4. Creamos las nuevas predicciones. Si la probabilidad es > 0.30, predecimos 1 (Churn), si no, 0.
nuevas_predicciones = (prob_churn_yes >= nuevo_umbral).astype(int)

# 5. Imprimir el NUEVO Reporte de Clasificación con el umbral ajustado
print("\n--- Reporte de Clasificación con Umbral de 0.30 ---")
print(classification_report(y_test, nuevas_predicciones))

--- ANTES DE LA TRANSFORMACIÓN ---
  InternetService        Contract     PaymentMethod
0             DSL  Month-to-month  Electronic check
1             DSL        One year      Mailed check
2             DSL  Month-to-month      Mailed check


--- DESPUÉS DE LA TRANSFORMACIÓN ---
   InternetService_Fiber optic  InternetService_No  Contract_One year  \
0                        False               False              False   
1                        False               False               True   
2                        False               False              False   

   Contract_Two year  
0              False  
1              False  
2              False  
--- TAMAÑO DE LOS CONJUNTOS ---
Características de entrenamiento (X_train): (5634, 13601)
Características de prueba (X_test):          (1409, 13601)
Objetivo de entrenamiento (y_train):    (5634,)
Objetivo de prueba (y_test):         (1409,)
Entrenando el modelo Random Forest... ¡Esto puede tardar unos segundos!

¡Modelo entrenado 